# 1. 데이터 로드


In [2]:
# 데이터 로드
import pandas as pd
import numpy as np
# 데이터 불러오기 (불필요한 인덱스 컬럼 제거)
sales_df = pd.read_csv('Bakery sales.csv', index_col=0)

## 2. 데이터 전처리


###    2.1 데이터 분석에 필요한 컬럼 추가


In [5]:
# 날짜 컬럼을 datetime 형식으로 변환
sales_df['date'] = pd.to_datetime(sales_df['date'])

In [6]:
# 'time' 컬럼에서 시(hour) 정보 추출 (시간대별 분석을 위해 추가)
sales_df['hour'] = pd.to_datetime(sales_df['time'] + ':00', format='%H:%M:%S').dt.hour

In [7]:
# 시간대 컬럼 추가 (08 ~ 11시 : 아침, 11 ~ 14시 : 점심, 14시 이후 : 오후)
sales_df['part_of_day'] = np.where(
    sales_df['hour'] < 11, '아침',
    np.where(sales_df['hour'] < 14, '점심', '오후')
)

In [8]:
# sale 데이터 셋의 개별 가격의 데이터 전처리
sales_df['unit_price']=sales_df['unit_price'].str.strip('€')
sales_df['unit_price']=sales_df['unit_price'].replace(',','.',regex=True)
sales_df['unit_price']=sales_df['unit_price'].astype('float64')

In [9]:
# 각 제품 주문 별 주문가격(개별가격 * 수량) 컬럼 추가
sales_df['total_price']=sales_df['unit_price']*sales_df['Quantity']

In [10]:
# 요일 별 판매량 분석을 위한 전처리 (0: 월 ~ 6: 일)
sales_df['weekday'] = sales_df['date'].dt.weekday

In [11]:
# 공휴일 분류를 위한 컬럼 추가
sales_df['month_day']=sales_df['date'].dt.strftime('%m-%d')

###    2.2환불 데이터 전처리

In [13]:
# 환불 데이터 전처리에 필요한 index컬럼 추가
sales_df=sales_df.reset_index()

In [14]:
# refund_data 에 환불 내역 저장         #1295건
refund_data=sales_df[sales_df['Quantity']<=0]

In [15]:
# 환불 데이터를 기준으로 구매 데이터와 병합
merged_data = pd.merge(sales_df[sales_df['Quantity'] >= 0], refund_data, on=['date','article'], how='inner', suffixes=('', '_refund'))

In [16]:
# 병합된 데이터에서 환불 시간대보다 구매 시간대가 적거나 같고, 구매 금액과 환불금액이 같은 데이터를 refund 컬럼에서 1로 출력
merged_data['refund'] = np.where((merged_data['hour'] <= merged_data['hour_refund'])&(merged_data['total_price']==abs(merged_data['total_price_refund'])), 1, 0)

In [17]:
# refund 컬럼 값이 1인 데이터만 저장
merged_data=merged_data[merged_data['refund']==1]

In [18]:
# 데이터를 주문번호순으로 정렬 후 index와 index_refund 중복값 제거 
merged_unique_refund=merged_data.sort_values('ticket_number_refund').drop_duplicates(subset='index', keep='first').drop_duplicates(subset='index_refund', keep='last')

In [19]:
# 구매데이터와 환불 데이터 index를 list 형태로 저장
refund_data_list = np.concatenate([merged_unique_refund['index'], merged_unique_refund['index_refund']])

In [20]:
# 전처리를 한번더 진행하기 위해서 우선 구매와 환불이 일치하는 데이터들은 기존 데이터 셋에서 삭제
sale=sales_df[~sales_df['index'].isin(refund_data_list)]
refund1=sales_df[sales_df['index'].isin(refund_data_list)]

In [21]:
# 남은 환불 데이터 건을 refund_data2에 저장
refund_data2=sale[sale['Quantity']<=0]

In [22]:
# 환불 데이터를 기준으로 구매 데이터와 병합
merged_data2 = pd.merge(sale[sale['Quantity'] >= 0], refund_data2, on=['date','article'], how='inner', suffixes=('', '_refund'))

In [23]:
# 병합된 데이터에서 환불 시간대보다 구매 시간대가 적거나 같고, 구매 금액과 환불금액이 같은 데이터를 refund 컬럼에서 1로 출력
merged_data2['refund'] = np.where((merged_data2['hour'] <= merged_data2['hour_refund'])&(merged_data2['total_price']==abs(merged_data2['total_price_refund'])), 1, 0)

In [24]:
# refund 컬럼값이 1인 데이터만 저장
merged_data2=merged_data2[merged_data2['refund']==1]

In [25]:
#  데이터를 주문번호순으로 정렬후 index와 index_refund 중복값 제거 
merged_unique_refund2=merged_data2.sort_values('ticket_number_refund').drop_duplicates(subset='index', keep='first').drop_duplicates(subset='index_refund', keep='last')

In [26]:
# 구매데이터와 환불 데이터 index를 list 형태로 저장
refund_data_list2 = np.concatenate([merged_unique_refund2['index'], merged_unique_refund2['index_refund']])


In [27]:
sale2=sale[~sale['index'].isin(refund_data_list2)]
refund2=sale[sale['index'].isin(refund_data_list2)]

In [28]:
# 환불 데이터 삭제시 사용하였던 인덱스 컬럼 삭제
sale_df=sale2.drop(columns='index',axis=1)

###    2.3 매출이 저조한 제품 전처리

In [34]:
sale_rate=sale_df.groupby('article',as_index=False)\
.agg(total_order=('Quantity','sum')).sort_values('total_order',ascending=False)

In [35]:
# 파레토 법칙을 적용하여 누적 비율 상위 80% 이외의 제품 전처리
sale_rate['rate']=(sale_rate['total_order']/sum(sale_rate['total_order']) )*100
pareto=sale_rate.head(16)

In [39]:
sale_df=sale_df[sale_df['article'].isin(pareto['article'])]

###    2.4 공휴일 데이터 전처리


#### 프랑스 국가 공휴일 리스트
01-01 신정 (휴무)<br>
05-01 노동절 (1시까지 영업)<br>
05-08 승전의 날 (1시까지 영업)<br>
05-09 예수 승천의날 (1시까지 영업)<br>
05-20 성모 강림의 날 (휴무)<br>
07-14 혁명 기념일 (1시까지 영업)<br>
08-15 성모 승천의 날 (1시까지 영업) <br>
11-01 만성절 (2시까지 영업)<br>
11-11 휴전의 날(2시까지 영업)<br>
12-25 크리스마스 (1시까지 영업)<br>
12-26 성 스테판의 날 (1시까지 영업)<br>

In [74]:
# 공휴일 리스트 작성
holiday_list=['01-01','05-01','05-08','05-09','05-20','07-14','08-15','11-01','11-11','12-25','12-26']
# renewal 데이터 셋 복사
holiday_sale=sale_df.copy()
# 공휴일 데이터 구분을 위한 임시 컬럼 추가
holiday_sale['month_day']=holiday_sale['date'].dt.strftime('%m-%d')
holiday_sale['holiday']=np.where(holiday_sale['month_day'].isin(holiday_list),1,0)

In [76]:
holiday_data=holiday_sale[holiday_sale['holiday']==1]
sevenday_data=holiday_sale[holiday_sale['holiday']==0]

In [78]:
# 일요일 데이터를 holiday_data에 포함시키기
sunday_data = sevenday_data[sevenday_data['weekday'] == 6]  # 일요일은 요일 코드가 6
sixday_data = sevenday_data[sevenday_data['weekday'] != 6]  # 일요일 제거
holiday_data = pd.concat([holiday_data, sunday_data], ignore_index=True)

In [80]:
sixday_sales_df=sixday_data.drop(columns='holiday')
weekly_sales_df=sevenday_data.drop(columns='holiday')
holiday_sales_df=holiday_data.drop(columns='holiday')

In [82]:
# 월 ~ 토요일 데이터 세트 생성
sixday_sales_df.to_csv("C:/easy_python/bakery/sixday_sales_df.csv", index=False)
# 월 ~ 일요일 데이터 세트 생성
weekly_sales_df.to_csv("C:/easy_python/bakery/weekly_sales_df.csv", index=False)
# 일 + 공휴일 데이터 세트 생서
holiday_sales_df.to_csv("C:/easy_python/bakery/holiday_sales_df.csv", index=False)

In [84]:
holiday_sale['article'].unique()

array(['BAGUETTE', 'PAIN AU CHOCOLAT', 'TRADITIONAL BAGUETTE',
       'CROISSANT', 'BANETTE', 'SPECIAL BREAD', 'COUPE', 'BOULE 400G',
       'CAMPAGNE', 'CEREAL BAGUETTE', 'COMPLET', 'COOKIE', 'VIK BREAD',
       'FORMULE SANDWICH', 'TARTELETTE', 'ECLAIR'], dtype=object)